In [4]:
# This notebook takes as input GOI_out_AA.csv files (from getMutationCounts_overall_and_GOI.py),
# patient metadata, seurat metadata, fusionsDF, and creates a BY CELL 
# summaryTable. The goal with this table is to provide an answer to questions like
# 'which patients have which mutations?', and 'how many cells have clinically relevant
# mutations?' 

In [5]:
import summarizeLib
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # want to disable this SettingWithCopyWarning

In [6]:
# READ IN ALL OF THESE BY-GENE AMINO-ACID LEVEL MUTATION COUNTS OBJECTS
mutsPATH = '/Users/lincoln.harris/code/SNP_calling_pipeline/getMutationCounts/'
egfrPATH = mutsPATH + 'egfr_germline_out_AA.csv'
brafPATH = mutsPATH + 'braf_germline_out_AA.csv'
krasPATH = mutsPATH + 'kras_germline_out_AA.csv'

egfr_df = pd.read_csv(egfrPATH, header=None, names=['cell', 'mutations'])
braf_df = pd.read_csv(brafPATH, header=None, names=['cell', 'mutations'])
kras_df = pd.read_csv(krasPATH, header=None, names=['cell', 'mutations'])
egfr_df

cell                                    mutations
0        K21_B003995                                           []
1     L22_1001000408                                    ['Q787Q']
2         A8_B001557                                           []
3        L22_B001016                                           []
4        C20_B002073                                           []
5         D8_B001474                                           []
6         C5_B002572                                           []
7        E17_B003116                                           []
8        A15_B000420                                           []
9        H10_B002573                                           []
10       I17_B002078                                           []
11       D20_B003518                                           []
12       M20_B003529                                           []
13        B2_B000901                                           []
14       C18_B002079                                           []
15        O8_B001545                                           []
16    H15_1001000408                                           []
17        M7_B003527                                           []
18       L14_B002077                                           []
19        G1_B002572                                           []
20       C22_B003588                                           []
21        A8_B000863                                           []
22       I19_B001007                                           []
23        M1_B000862                                    ['L387M']
24       K14_B003894                                           []
25    L14_1001000408                           ['T903T', 'T629T']
26        B2_B001464                                           []
27       G19_B003522                                           []
28       M14_B001008                           ['R521K', 'Q787Q']
29       N21_B001558                                           []
...              ...                                          ...
9851     N11_B001470                                           []
9852     E15_B003120                                           []
9853   G5_1001000408                                           []
9854     D13_B003527                                           []
9855     K17_B003527                                           []
9856      H9_B001007                                           []
9857     N22_B003775                                           []
9858     G18_B000420                                           []
9859     M18_B003112                                           []
9860     K17_B003894  ['E746_A750delELREA', 'E746_T751delELREAT']
9861      L1_B000863                                           []
9862     A15_B002078                                           []
9863     C18_B003116                                           []
9864      D4_B001558                                           []
9865   C2_1001000302                                           []
9866      E3_B003132                                           []
9867     J13_B003656                                           []
9868      O9_B001475                                           []
9869  E13_1001000408         ['R521K', 'Q787Q', 'T903T', 'T629T']
9870      M7_B003658                                           []
9871      L2_B003658                                           []
9872     N13_B000422                                           []
9873      D2_B002572                                           []
9874  I15_1001000339                                           []
9875      H4_B001558                                           []
9876     D17_B003777                                           []
9877      J7_B003648                                           []
9878     A11_B002077                                           []
9879      J8_B003129                                    

In [7]:
# FIRST STEP IS TO GENERATE THE mutationsDF
mutationsDF = pd.DataFrame(columns=['cell', 'brafMut', 'egfrMut', 'krasMut'])
mutationsDF['cell'] = egfr_df['cell']
mutationsDF['egfrMut'] = egfr_df['mutations'] # fill in EGFR first -- this is ok bc the cell order is based on egfr_df
summarizeLib.mutationsDF_fillIn('braf', braf_df, mutationsDF) 
summarizeLib.mutationsDF_fillIn('kras', kras_df, mutationsDF)
mutationsDF

cell brafMut                                      egfrMut  \
0        K21_B003995      []                                           []   
1     L22_1001000408      []                                    ['Q787Q']   
2         A8_B001557      []                                           []   
3        L22_B001016      []                                           []   
4        C20_B002073      []                                           []   
5         D8_B001474      []                                           []   
6         C5_B002572      []                                           []   
7        E17_B003116      []                                           []   
8        A15_B000420      []                                           []   
9        H10_B002573      []                                           []   
10       I17_B002078      []                                           []   
11       D20_B003518      []                                           []   
12       M20_B003529      []                                           []   
13        B2_B000901      []                                           []   
14       C18_B002079      []                                           []   
15        O8_B001545      []                                           []   
16    H15_1001000408      []                                           []   
17        M7_B003527      []                                           []   
18       L14_B002077      []                                           []   
19        G1_B002572      []                                           []   
20       C22_B003588      []                                           []   
21        A8_B000863      []                                           []   
22       I19_B001007      []                                           []   
23        M1_B000862      []                                    ['L387M']   
24       K14_B003894      []                                           []   
25    L14_1001000408      []                           ['T903T', 'T629T']   
26        B2_B001464      []                                           []   
27       G19_B003522      []                                           []   
28       M14_B001008      []                           ['R521K', 'Q787Q']   
29       N21_B001558      []                                           []   
...              ...     ...                                          ...   
9851     N11_B001470      []                                           []   
9852     E15_B003120      []                                           []   
9853   G5_1001000408      []                                           []   
9854     D13_B003527      []                                           []   
9855     K17_B003527      []                                           []   
9856      H9_B001007      []                                           []   
9857     N22_B003775      []                                           []   
9858     G18_B000420      []                                           []   
9859     M18_B003112      []                                           []   
9860     K17_B003894      []  ['E746_A750delELREA', 'E746_T751delELREAT']   
9861      L1_B000863      []                                           []   
9862     A15_B002078      []                                           []   
9863     C18_B003116      []                                           []   
9864      D4_B001558      []                                           []   
9865   C2_1001000302      []                                           []   
9866      E3_B003132      []                                           []   
9867     J13_B003656      []                                           []   
9868      O9_B001475      []                                           []   
9869  E13_1001000408      []         ['R521K', 'Q787Q', 'T903T', 'T629T']   
9870      M7_B003658      []                                           []   
9871      L2

In [8]:
# CONVERTING LISTS INTO STRS. MAKES DOWNSTEAM ANALYSIS EASIER
summarizeLib.removeExtraCharacters_mutationsDF('egfr', mutationsDF)
summarizeLib.removeExtraCharacters_mutationsDF('braf', mutationsDF)
summarizeLib.removeExtraCharacters_mutationsDF('kras', mutationsDF)
mutationsDF

cell brafMut                               egfrMut krasMut
0        K21_B003995                                                      
1     L22_1001000408                                         Q787Q        
2         A8_B001557                                                      
3        L22_B001016                                                      
4        C20_B002073                                                      
5         D8_B001474                                                      
6         C5_B002572                                                      
7        E17_B003116                                                      
8        A15_B000420                                                      
9        H10_B002573                                                      
10       I17_B002078                                                      
11       D20_B003518                                                      
12       M20_B003529                                                      
13        B2_B000901                                                      
14       C18_B002079                                                      
15        O8_B001545                                                      
16    H15_1001000408                                                      
17        M7_B003527                                                      
18       L14_B002077                                                      
19        G1_B002572                                                      
20       C22_B003588                                                      
21        A8_B000863                                                      
22       I19_B001007                                                      
23        M1_B000862                                         L387M        
24       K14_B003894                                                      
25    L14_1001000408                                   T903T,T629T        
26        B2_B001464                                                      
27       G19_B003522                                                      
28       M14_B001008                                   R521K,Q787Q        
29       N21_B001558                                                      
...              ...     ...                                   ...     ...
9851     N11_B001470                                                      
9852     E15_B003120                                                      
9853   G5_1001000408                                                      
9854     D13_B003527                                                      
9855     K17_B003527                                                  G13C
9856      H9_B001007                                                      
9857     N22_B003775                                                      
9858     G18_B000420                                                      
9859     M18_B003112                                                      
9860     K17_B003894          E746_A750delELREA,E746_T751delELREAT        
9861      L1_B000863                                                      
9862     A15_B002078                                                      
9863     C18_B003116                                                      
9864      D4_B001558                                                      
9865   C2_1001000302                                                      
9866      E3_B003132                                                      
9867     J13_B003656                                                      
9868      O9_B001475                                                      
9869  E13_1001000408                       R521K,Q787Q,T903T,T629T    A11A
9870      M7_B003658                                                      
9871      L2_B003658                                                      
9872     N13_B000422                     

In [9]:
# READ IN patientMetadata
patientMetadata = pd.read_csv('/Users/lincoln.harris/code/SNP_calling_pipeline/cDNA_plate_metadata.csv')
patientMetadata = patientMetadata.drop([0,1]) # first two rows are wierd
patientMetadata

plate sample_type    patient_id         DOB  gender  \
2     1001000332   cell_line         H1975         NaN     NaN   
3     1001000330   cell_line          TPH1         NaN     NaN   
4     1001000302  Lung_tumor         TH158  1959-11-23  Female   
5     1001000301  Lung_tumor         TH157  2016-11-25    Male   
6     1001000292  Lung_tumor         TH156  1973-03-17    Male   
7     1001000293  Lung_tumor         TH156  1973-03-17    Male   
8     1001000294  Lung_tumor         TH156  1973-03-17    Male   
9     1001000295  Lung_tumor         TH156  1973-03-17    Male   
10    1001000296  Lung_tumor         TH166  1974-12-15    Male   
11    1001000297  Lung_tumor         TH166  1974-12-15    Male   
12    1001000298  Lung_tumor         TH166  1974-12-15    Male   
13    1001000299  Lung_tumor         TH166  1974-12-15    Male   
14    1001000300  Lung_tumor         TH166  1974-12-15    Male   
15    1001000303  Lung_tumor     TH134_PDX  1969-04-17    Male   
16    1001000304  Lung_tumor     TH134_PDX  1969-04-17    Male   
17    1001000305  Lung_tumor     TH134_PDX  1969-04-17    Male   
18    1001000306  Lung_tumor     TH134_PDX  1969-04-17    Male   
19    1001000307  Lung_tumor     TH134_PDX  1969-04-17    Male   
20    1001000314  Lung_tumor         TH103  1971-07-27  Female   
21    1001000308  Lung_tumor        TH143B  1931-10-09    Male   
22    1001000315  Lung_tumor         TH103  1971-07-27  Female   
23    1001000327  Lung_tumor         TH103  1971-07-27  Female   
24    1001000328  Lung_tumor         TH103  1971-07-27  Female   
25    1001000329  Lung_tumor         TH103  1971-07-27  Female   
26    1001000316  Lung_tumor         TH103  1971-07-27  Female   
27    1001000317  Lung_tumor         TH103  1971-07-27  Female   
28    1001000320  Lung_tumor         TH103  1971-07-27  Female   
29    1001000322  Lung_tumor         TH103  1971-07-27  Female   
30    1001000326  Lung_tumor         TH103  1971-07-27  Female   
31   10001000325  Lung_tumor         TH103  1971-07-27  Female   
..           ...         ...           ...         ...     ...   
271      B001478     Exclude         TH257  1966-05-14    Male   
272      B001479     Exclude         TH257  1966-05-14    Male   
273      B001470  Lung_tumor         TH248  1965-05-04  Female   
274      B001474  Lung_tumor         TH248  1965-05-04  Female   
275      B001475  Lung_tumor         TH248  1965-05-04  Female   
276      B001460  Lung_tumor         TH266  1978-09-20  Female   
277      B001464  Lung_tumor         TH266  1978-09-20  Female   
278      B003093  Lung_tumor         TH266  1978-09-20  Female   
279      B003097  Lung_tumor         TH266  1978-09-20  Female   
280      B001565  Lung_tumor         TH231  1981-01-30    Male   
281      B003083  Lung_tumor         TH231  1981-01-30    Male   
282      B003087  Lung_tumor         TH231  1981-01-30    Male   
283      B001448  Lung_tumor         TH124  1964-03-11  Female   
284      B001449  Lung_tumor         TH124  1964-03-11  Female   
285      B001450  Lung_tumor         TH124  1964-03-11  Female   
286      B001509  Lung_tumor         TH124  1964-03-11  Female   
287      B001553  Lung_tumor  TH179_Normal  1955-11-05    Male   
288      B001554  Lung_tumor  TH179_Normal  1955-11-05    Male   
289      B001558  Lung_tumor  TH179_Normal  1955-11-05    Male   
290      B001560  Lung_tumor  TH179_Normal  1955-11-05    Male   
291      B001543  Lung_tumor         TH179  1955-11-05    Male   
292      B001545  Lung_tumor         TH179  1955-11-05    Male   
293      B001548  Lung_tumor         TH179  1955-11-05    Male   
294      B001551  Lung_tumor         TH179  1955-11-05    Male   
295      B001556  Lung_tumor         TH179  1955-11-05    Male   
296      B001557  Lung_tumor         TH179  1955-11-05    Male   
297      B001559  Lung_tumor         TH179  1955-11-05    Male   
298      B002571  Lung_tumor         TH266  1978-09-20  Female   
299      B002572  Lung_tumor         TH266  1978-09

In [10]:
# INIT THE SUMMARY TABLE
cols = ['cell', 'patient', 'clinical_driver_gene', 'clinical_mutation', 'coverage_to_ROI', 'clin_mut_found_bool', 'mutations_found_EGFR', 'mutations_found_BRAF', 'mutations_found_KRAS', 'fusions_found', 'tumorCell_bool']
summaryTable = pd.DataFrame(columns=cols)
summaryTable['cell'] = mutationsDF['cell']
summaryTable

cell patient clinical_driver_gene clinical_mutation  \
0        K21_B003995     NaN                  NaN               NaN   
1     L22_1001000408     NaN                  NaN               NaN   
2         A8_B001557     NaN                  NaN               NaN   
3        L22_B001016     NaN                  NaN               NaN   
4        C20_B002073     NaN                  NaN               NaN   
5         D8_B001474     NaN                  NaN               NaN   
6         C5_B002572     NaN                  NaN               NaN   
7        E17_B003116     NaN                  NaN               NaN   
8        A15_B000420     NaN                  NaN               NaN   
9        H10_B002573     NaN                  NaN               NaN   
10       I17_B002078     NaN                  NaN               NaN   
11       D20_B003518     NaN                  NaN               NaN   
12       M20_B003529     NaN                  NaN               NaN   
13        B2_B000901     NaN                  NaN               NaN   
14       C18_B002079     NaN                  NaN               NaN   
15        O8_B001545     NaN                  NaN               NaN   
16    H15_1001000408     NaN                  NaN               NaN   
17        M7_B003527     NaN                  NaN               NaN   
18       L14_B002077     NaN                  NaN               NaN   
19        G1_B002572     NaN                  NaN               NaN   
20       C22_B003588     NaN                  NaN               NaN   
21        A8_B000863     NaN                  NaN               NaN   
22       I19_B001007     NaN                  NaN               NaN   
23        M1_B000862     NaN                  NaN               NaN   
24       K14_B003894     NaN                  NaN               NaN   
25    L14_1001000408     NaN                  NaN               NaN   
26        B2_B001464     NaN                  NaN               NaN   
27       G19_B003522     NaN                  NaN               NaN   
28       M14_B001008     NaN                  NaN               NaN   
29       N21_B001558     NaN                  NaN               NaN   
...              ...     ...                  ...               ...   
9851     N11_B001470     NaN                  NaN               NaN   
9852     E15_B003120     NaN                  NaN               NaN   
9853   G5_1001000408     NaN                  NaN               NaN   
9854     D13_B003527     NaN                  NaN               NaN   
9855     K17_B003527     NaN                  NaN               NaN   
9856      H9_B001007     NaN                  NaN               NaN   
9857     N22_B003775     NaN                  NaN               NaN   
9858     G18_B000420     NaN                  NaN               NaN   
9859     M18_B003112     NaN                  NaN               NaN   
9860     K17_B003894     NaN                  NaN               NaN   
9861      L1_B000863     NaN                  NaN               NaN   
9862     A15_B002078     NaN                  NaN               NaN   
9863     C18_B003116     NaN                  NaN               NaN   
9864      D4_B001558     NaN                  NaN               NaN   
9865   C2_1001000302     NaN                  NaN               NaN   
9866      E3_B003132     NaN                  NaN               NaN   
9867     J13_B003656     NaN                  NaN               NaN   
9868      O9_B001475     NaN                  NaN               NaN   
9869  E13_1001000408     NaN                  NaN               NaN   
9870      M7_B003658     NaN                  NaN               NaN   
9871      L2_B003658     NaN                  NaN               NaN   
9872     N13_B000422     NaN                  NaN               NaN   
9873      D2_B002572     NaN                  NaN               NaN   
9874  I15_1001000339     NaN                  NaN               NaN   
9875      H4_B001558     NaN            

In [11]:
# FILL IN VARIOUS METADATA COLS
summarizeLib.genericSummaryTableFillIn('patient_id', 'patient', summaryTable, patientMetadata)
summarizeLib.genericSummaryTableFillIn('driver_gene', 'clinical_driver_gene', summaryTable, patientMetadata)
summarizeLib.genericSummaryTableFillIn('driver_mutation', 'clinical_mutation', summaryTable, patientMetadata)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                 NaN   
1                       L858R             NaN                 NaN   
2                       V600E             NaN                 NaN   
3                         NaN             NaN                 NaN   
4           

In [12]:
# FILL IN MUTATIONS FOUND COL 
summaryTable['mutations_found_EGFR'] = mutationsDF['egfrMut']
summaryTable['mutations_found_KRAS'] = mutationsDF['krasMut']
summaryTable['mutations_found_BRAF'] = mutationsDF['brafMut']
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                 NaN   
1                       L858R             NaN                 NaN   
2                       V600E             NaN                 NaN   
3                         NaN             NaN                 NaN   
4           

In [13]:
# READ IN FUSIONS DATAFRAME, THEN FILL IN summaryTable
fusionsDF = pd.read_csv('/Users/lincoln.harris/code/SNP_calling_pipeline/summaryTable/fusion_dataframe.csv')
fusionsDF

ALK--EML4          ALK_any         EML4_any     NTRK_any  \
0        C2_B000862      K14_B003132      L18_B003120   C8_B003191   
1        P1_B001464      G19_B003121      D10_B003523   D8_B003126   
2       M11_B003522   O13_1001000339      I22_B000276  J11_B003528   
3     G8_1001000317      I13_B003046       A4_B001607   B5_B002078   
4    A7_10001000325       M8_B003119       I6_B003642  J18_B002571   
5       M12_B003522       K2_B003522       I4_B001607   P6_B001545   
6   B11_10001000325       N8_B001556      P20_B002571  O17_B002097   
7        G2_B000862      D18_B002073       E5_B001545  K10_B003528   
8       J15_B000862   O17_1001000409       E1_B003117  C17_B003071   
9     E7_1001000317       C2_B000862       O2_B003067          NaN   
10      M21_B003518       P1_B001464       C2_B000862          NaN   
11       N4_B001464      M11_B003522       P1_B001464          NaN   
12              NaN       C7_B003523       O6_B000420          NaN   
13              NaN    G8_1001000317      M11_B003522          NaN   
14              NaN   A7_10001000325       O2_B001543          NaN   
15              NaN      M12_B003522    G8_1001000317          NaN   
16              NaN       P8_B003191   A7_10001000325          NaN   
17              NaN      P13_B003112   K22_1001000410          NaN   
18              NaN       I7_B001475      M12_B003522          NaN   
19              NaN  B11_10001000325      D14_B003776          NaN   
20              NaN      A13_B001543  B11_10001000325          NaN   
21              NaN       J5_B003529       M5_B003648          NaN   
22              NaN       M3_B003648    D8_1001000408          NaN   
23              NaN      K18_B003071   L16_1001000410          NaN   
24              NaN   E19_1001000407      H17_B003115          NaN   
25              NaN      C11_B003071      E18_B002573          NaN   
26              NaN       P1_B001474      L21_B003132          NaN   
27              NaN      P22_B003776       O8_B003648          NaN   

        RET_any        ROS1_any  
0    E2_B003920     D10_B003523  
1   B15_B003529     L17_B003116  
2   M21_B002572      F5_B001556  
3    J6_B001470      I5_B003070  
4   J19_B003070      J5_B003125  
5   G13_B000862   G5_1001000327  
6   D15_B002078     J22_B003120  
7   P19_B002572  O24_1001000377  
8   D19_B003132      A3_B003067  
9   G11_B001543     L20_B002572  
10   D8_B003527  O23_1001000377  
11          NaN      A6_B003132  
12          NaN   H7_1001000377  
13          NaN      A1_B003067  
14          NaN     N16_B003132  
15          NaN      L3_B003070  
16          NaN      J7_B003067  
17          NaN     E10_B003528  
18          NaN      F7_B003126  
19          NaN     M19_B003777  
20          NaN      N2_B003129  
21          NaN     J14_B003116  
22          NaN      G5_B003659  
23          NaN      B3_B003187  
24          NaN     M18_B003120  
25          NaN     H19_B003120  
26          NaN      H2_B003126  
27          NaN     E11_B003119

In [14]:
summarizeLib.fusionsFillIn(fusionsDF, summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                 NaN   
1                       L858R             NaN                 NaN   
2                       V600E             NaN                 NaN   
3                         NaN             NaN                 NaN   
4           

In [15]:
# SET UP A COL TO TRANSLATE 'RAW' MUTATION CALLS TO 'CLINICAL'
summaryTable['mutations_found_translated'] = ""
summarizeLib.translatedMutsFillIn_EGFR(summaryTable)
summarizeLib.translatedMutsFillIn_nonEGFR('KRAS', summaryTable)
summarizeLib.translatedMutsFillIn_nonEGFR('BRAF', summaryTable)
summarizeLib.translatedMutsFillIn_fusions(summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                 NaN   
1                       L858R             NaN                 NaN   
2                       V600E             NaN                 NaN   
3                         NaN             NaN                 NaN   
4           

In [16]:
# CONVERT LISTS TO STRING, SO I CAN GET SET -- probably not necessary, actually 
summarizeLib.convertToString(summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                 NaN   
1                       L858R             NaN                 NaN   
2                       V600E             NaN                 NaN   
3                         NaN             NaN                 NaN   
4           

In [17]:
# FILL IN clin_mut_found_bool
summarizeLib.clinMutFound_fillIn(summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                   0   
1                       L858R             NaN                   0   
2                       V600E             NaN                   0   
3                         NaN             NaN                   0   
4           

In [18]:
# FILL IN  tumorCellBool
summarizeLib.tumorCellBoolFillIn(summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19             NaN                   0   
1                       L858R             NaN                   0   
2                       V600E             NaN                   0   
3                         NaN             NaN                   0   
4           

In [20]:
# GET PER-CELL ROI COVERAGE DFs
braf_V600E_cov_nonZero = summarizeLib.getNonZeroCovROI('braf', 'V600E')
egfr_L858R_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'L858R')
egfr_exon19del_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'exon19del')
egfr_exon20ins_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'exon20ins') # this guy is totally empty...
egfr_G719X_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'G719X')
egfr_L861Q_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'L861Q')
egfr_S768I_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'S768I')
egfr_T790M_cov_nonZero = summarizeLib.getNonZeroCovROI('egfr', 'T790M')
kras_G12C_cov_nonZero = summarizeLib.getNonZeroCovROI('kras', 'G12C')
kras_G13X_cov_nonZero = summarizeLib.getNonZeroCovROI('kras', 'G13X')
kras_Q61X_cov_nonZero = summarizeLib.getNonZeroCovROI('kras', 'Q61X')
egfr_L858R_cov_nonZero

cellName  coverage_bool_vcf  depth_vcf  coverage_bool_gvcf  \
66    A13_1001000407                  1         12                   1   
67    A13_1001000408                  0          0                   1   
121   A15_1001000408                  0          0                   1   
153   A16_1001000407                  1          6                   1   
155   A16_1001000412                  1         16                   1   
185   A17_1001000407                  0          0                   1   
212   A18_1001000408                  0          0                   1   
285   A20_1001000408                  1          6                   1   
384    A3_1001000412                  0          0                   1   
542   B10_1001000412                  1          3                   1   
588   B12_1001000412                  1          2                   1   
607   B13_1001000407                  0          0                   1   
609   B13_1001000412                  1          3                   1   
660   B15_1001000407                  0          0                   1   
730   B17_1001000407                  1          6                   1   
732   B17_1001000412                  0          0                   1   
783   B19_1001000407                  1         17                   1   
824   B20_1001000407                  0          0                   1   
875   B22_1001000407                  1          8                   1   
876   B22_1001000408                  0          0                   1   
974    B5_1001000412                  1          3                   1   
1089  C10_1001000407                  1         12                   1   
1114  C11_1001000407                  1         10                   1   
1115  C11_1001000408                  1         10                   1   
1160  C13_1001000408                  1          3                   1   
1278  C17_1001000408                  0          0                   1   
1310  C18_1001000407                  1          3                   1   
1453  C22_1001000407                  1          4                   1   
1454  C22_1001000408                  1          2                   1   
1487   C2_1001000412                  0          0                   1   
...              ...                ...        ...                 ...   
7684  N12_1001000407                  1          3                   1   
7770  N15_1001000412                  0          0                   1   
7838  N17_1001000408                  1          3                   1   
7897  N19_1001000407                  1          9                   1   
7944  N20_1001000407                  1          2                   1   
7945  N20_1001000408                  1          3                   1   
8043   N3_1001000407                  0          0                   1   
8110   N6_1001000407                  1          2                   1   
8207  O10_1001000407                  1          3                   1   
8230  O11_1001000412                  1          2                   1   
8251  O12_1001000407                  0          0                   1   
8301  O14_1001000408                  1          3                   1   
8370  O16_1001000412                  1          6                   1   
8404  O17_1001000412                  1          7                   1   
8563  O22_1001000408                  0          0                   1   
8564  O22_1001000412                  0          0                   1   
8635   O4_1001000407                  1          4                   1   
8726   O8_1001000412                  0          0                   1   
8775  P10_1001000408                  1          4                   1   
8847  P13_1001000407                  1          6                   1   
8906  P15_1001000412                  1          7                   1   
9017  P19_1001000412                  1          2                   1   
9090  P21_100100

In [21]:
# FIX UP SOME OF THE WEIRD ONES
kras_G13X_cov_nonZero['depth_gvcf'][4202] = 34	
kras_Q61X_cov_nonZero['depth_gvcf'][6431] = 92
egfr_exon19del_cov_nonZero['depth_gvcf'] = egfr_exon19del_cov_nonZero['depth_gvcf'].str.strip('[')
egfr_exon19del_cov_nonZero['depth_gvcf'] = egfr_exon19del_cov_nonZero['depth_gvcf'].str.strip(']')
egfr_exon19del_cov_nonZero['depth_gvcf'] = egfr_exon19del_cov_nonZero['depth_gvcf'].str.strip("'")
egfr_exon19del_cov_nonZero

cellName  coverage_bool_vcf depth_vcf  coverage_bool_gvcf  \
0     A10_1001000407                  0         0                   0   
1     A10_1001000408                  0         0                   0   
2     A10_1001000412                  0         0                   0   
3        A10_B000863                  0         0                   0   
4        A10_B001007                  0         0                   0   
5        A10_B001470                  0         0                   0   
6        A10_B001474                  0         0                   0   
7        A10_B001545                  0         0                   0   
8        A10_B001548                  0         0                   0   
9        A10_B001554                  0         0                   0   
10       A10_B001558                  0         0                   0   
11       A10_B002572                  0         0                   0   
12       A10_B003070                  0         0                   0   
13       A10_B003071                  0         0                   0   
14       A10_B003115                  0         0                   0   
15       A10_B003125                  0         0                   0   
16       A10_B003518                  0         0                   0   
17       A10_B003523                  0         0                   0   
18       A10_B003527                  0         0                   0   
19       A10_B003528                  0         0                   0   
20       A10_B003529                  0         0                   0   
21       A10_B003588                  0         0                   0   
22    A11_1001000407                  0         0                   0   
23    A11_1001000408                  0         0                   0   
24    A11_1001000412                  0         0                   0   
25       A11_B000860                  0         0                   0   
26       A11_B000861                  0         0                   0   
27       A11_B000862                  0         0                   0   
28       A11_B001007                  0         0                   0   
29       A11_B001470                  0         0                   0   
...              ...                ...       ...                 ...   
9318      P8_B003518                  0         0                   0   
9319      P8_B003527                  0         0                   0   
9320      P8_B003648                  0         0                   0   
9321      P8_B003658                  0         0                   0   
9322      P8_B003995                  0         0                   0   
9323   P9_1001000407                  0         0                   0   
9324   P9_1001000408                  0         0                   0   
9325      P9_B000281                  0         0                   0   
9326      P9_B000419                  0         0                   0   
9327      P9_B000861                  0         0                   0   
9328      P9_B000862                  0         0                   0   
9329      P9_B001007                  0         0                   0   
9330      P9_B001470                  0         0                   0   
9331      P9_B001475                  0         0                   0   
9332      P9_B001543                  0         0                   0   
9333      P9_B001545                  0         0                   0   
9334      P9_B001548                  0         0                   0   
9335      P9_B001554                  0         0                   0   
9336      P9_B001558                  0         0                   0   
9337      P9_B002573                  0         0                   0   
9338      P9_B003067                  0         0                   0   
9339      P9_B003071                  0         0                   0   
9340      P9_B003187                  0         0                   0 

In [22]:
# FILL IN ROI COVERAGE TO SUMMARY TABLE
summarizeLib.ROI_coverage_fillIn(braf_V600E_cov_nonZero, 'BRAF', 'V600E', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_G719X_cov_nonZero, 'EGFR', 'G719X', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_L858R_cov_nonZero, 'EGFR', 'L858R', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_L861Q_cov_nonZero, 'EGFR', 'L861Q', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_S768I_cov_nonZero, 'EGFR', 'S768I', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_T790M_cov_nonZero, 'EGFR', 'T790M', summaryTable)
summarizeLib.ROI_coverage_fillIn(kras_G12C_cov_nonZero, 'KRAS', 'G12C', summaryTable)
summarizeLib.ROI_coverage_fillIn(kras_G13X_cov_nonZero, 'KRAS', 'G13X', summaryTable)
summarizeLib.ROI_coverage_fillIn(kras_Q61X_cov_nonZero, 'KRAS', 'Q61X', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_exon19del_cov_nonZero, 'EGFR', 'del19', summaryTable)
summarizeLib.ROI_coverage_fillIn(egfr_exon20ins_cov_nonZero, 'EGFR', 'ins20', summaryTable)
summaryTable

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation coverage_to_ROI clin_mut_found_bool  \
0                       del19               0                   0   
1                       L858R               0                   0   
2                       V600E               0                   0   
3                         NaN             NaN                   0   
4           

In [24]:
# TRIM IT DOWN
summaryTable_trimmed = summaryTable[['cell', 'patient', 'clinical_driver_gene', 'clinical_mutation', 'coverage_to_ROI', 'clin_mut_found_bool', 'tumorCell_bool', 'mutations_found_translated']]
summaryTable_trimmed.columns = ['cell', 'patient', 'clinical_driver_gene', 'clinical_mutation', 'coverage_to_ROI', 'clinical_mutation_found_bool', 'tumorCell_bool', 'mutations_found']
summaryTable_trimmed = summaryTable_trimmed[['cell', 'patient', 'clinical_driver_gene', 'clinical_mutation', 'mutations_found', 'coverage_to_ROI', 'clinical_mutation_found_bool', 'tumorCell_bool']]
summaryTable_trimmed

cell       patient clinical_driver_gene  \
0        K21_B003995         TH236                 EGFR   
1     L22_1001000408         TH185                 EGFR   
2         A8_B001557         TH179                 BRAF   
3        L22_B001016           NaN                  NaN   
4        C20_B002073  TH238_Normal                 BRAF   
5         D8_B001474         TH248                 EGFR   
6         C5_B002572         TH266                  ALK   
7        E17_B003116         TH231                  ALK   
8        A15_B000420         TH238                 BRAF   
9        H10_B002573         TH266                  ALK   
10       I17_B002078         TH238                 BRAF   
11       D20_B003518         TH171                  ALK   
12       M20_B003529         TH171                  ALK   
13        B2_B000901         TH218                 EGFR   
14       C18_B002079         TH238                 BRAF   
15        O8_B001545         TH179                 BRAF   
16    H15_1001000408         TH185                 EGFR   
17        M7_B003527         TH171                  ALK   
18       L14_B002077         TH238                 BRAF   
19        G1_B002572         TH266                  ALK   
20       C22_B003588         TH171                  ALK   
21        A8_B000863         TH220                  ALK   
22       I19_B001007           NaN                  NaN   
23        M1_B000862         TH220                  ALK   
24       K14_B003894         TH236                 EGFR   
25    L14_1001000408         TH185                 EGFR   
26        B2_B001464         TH266                  ALK   
27       G19_B003522         TH171                  ALK   
28       M14_B001008           NaN                  NaN   
29       N21_B001558  TH179_Normal                  ALK   
...              ...           ...                  ...   
9851     N11_B001470         TH248                 EGFR   
9852     E15_B003120         TH231                  ALK   
9853   G5_1001000408         TH185                 EGFR   
9854     D13_B003527         TH171                  ALK   
9855     K17_B003527         TH171                  ALK   
9856      H9_B001007           NaN                  NaN   
9857     N22_B003775         TH248                 EGFR   
9858     G18_B000420         TH238                 BRAF   
9859     M18_B003112         TH231                  ALK   
9860     K17_B003894         TH236                 EGFR   
9861      L1_B000863         TH220                  ALK   
9862     A15_B002078         TH238                 BRAF   
9863     C18_B003116         TH231                  ALK   
9864      D4_B001558  TH179_Normal                  ALK   
9865   C2_1001000302         TH158                 EGFR   
9866      E3_B003132         TH226                 EGFR   
9867     J13_B003656         TH227                  ALK   
9868      O9_B001475         TH248                 EGFR   
9869  E13_1001000408         TH185                 EGFR   
9870      M7_B003658         TH225                 KRAS   
9871      L2_B003658         TH225                 KRAS   
9872     N13_B000422         TH238                 BRAF   
9873      D2_B002572         TH266                  ALK   
9874  I15_1001000339         TH169                 EGFR   
9875      H4_B001558  TH179_Normal                  ALK   
9876     D17_B003777         TH248                 EGFR   
9877      J7_B003648         TH067                 EGFR   
9878     A11_B002077         TH238                 BRAF   
9879      J8_B003129         TH226                 EGFR   
9880  P22_1001000339         TH169                 EGFR   

            clinical_mutation  \
0                       del19   
1                       L858R   
2                       V600E   
3                         NaN   
4                       V600E   
5                       del19   
6                      fusion   
7                      fusion   
8                       V600E   
9                      fusi

In [ ]:
# write this fucker
summaryTable_trimmed.to_csv('/Users/lincoln.harris/Desktop/validationTable_TEST.csv', index=False)